In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import re

In [2]:
tic = pd.read_csv("TIC82_notnull_tic_kic.csv")

In [3]:
prot_mcq = pd.read_csv("prot_mcq_2014.dat",sep="\s+")

In [4]:
mcq_tic = pd.merge(prot_mcq, tic, how='inner', on='KIC')

mcq_tic

In [11]:
# wright et al 2018 (eqn 5): valid for range 1.1 < V-K < 7.0
def taucVK(V,K):
    if np.fabs(V - K - 4.05) <= 2.95:
        return pow(10.,0.64 + 0.25 * (V - K))
    else:
        return np.nan

def RoVK(Prot, V, K):
    return Prot / taucVK(V,K)
    
    
# wright et al 2018 (eqn 6): valid for range 0.08 < M/Msol < 1.36
#M in solar masses
def taucM(M):
    if np.fabs(M - 0.72) <= 0.64:
        return pow(10., 2.33 - 1.50 * M + 0.31 * pow(M, 2.))
    else:
        return np.nan
    
def RoM(Prot, M):
    return Prot / taucM(M)

def RoAvg(RoVK, RoM):
    if not np.isnan(RoVK) and not np.isnan(RoM):
        return (RoVK + RoM) / 2.
    elif np.isnan(RoVK) and not np.isnan(RoM):
        return RoM
    elif not np.isnan(RoVK) and np.isnan(RoM):
        return RoVK
    else:
        return np.nan

In [12]:
mcq_tic['RoVK'] = mcq_tic.apply(lambda x: RoVK(x['PRot'], x['Vmag'], x['Kmag']), axis=1)
mcq_tic['TaucVK'] = mcq_tic.apply(lambda x: taucVK(x['Vmag'], x['Kmag']), axis=1)
mcq_tic['RoM'] = mcq_tic.apply(lambda x: RoM(x['PRot'], x['Mass_x']), axis=1)
mcq_tic['TaucM'] = mcq_tic.apply(lambda x: taucM(x['Mass_x']), axis=1)

In [18]:
mcq_tic

,KIC,Teff_x,log(g),Mass_x,PRot,e_PRot,RPer,LPH,w,DC,...,e_Vmag,ID,Kmag,m_TIC,q_2MASS,Vmag,RoVK,TaucVK,RoM,TaucM
0,892376,3813,4.47,0.4699,1.532,0.007,7306.69,0.823,0.4503,0,...,0.080,279839336,10.721,NaN,AAA-222-111-000-0-0,14.580,0.038064,40.248528,0.031021,49.385546
1,1026146,4261,4.57,0.6472,14.891,0.120,11742.56,1.405,0.7229,0,...,0.172,279839149,12.839,NaN,AAA-222-111-000-0-0,15.627,0.685368,21.727012,0.482920,30.835361
2,1026474,4122,4.56,0.5914,1.569,0.006,30471.80,1.204,0.6061,0,...,0.149,280724397,12.382,NaN,AAA-222-111-000-0-0,15.824,0.049559,31.659205,0.044086,35.589651
3,1162635,3760,4.77,0.4497,15.678,0.019,10207.47,0.978,0.5445,1,...,0.309,282595905,12.674,NaN,AAA-222-111-000-0-0,16.408,0.418592,37.454155,0.300019,52.256621
4,1164102,4045,4.62,0.5606,31.496,0.474,5139.74,0.568,0.3939,0,...,0.137,285289877,12.430,NaN,AAA-222-111-000-0-0,15.852,1.006365,31.296805,0.816077,38.594412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34025,12934465,4714,4.88,0.7591,34.910,1.692,4572.33,0.433,0.2969,0,...,0.115,270628951,13.298,NaN,AAA-222-111-000-0-0,15.966,1.721670,20.276827,1.489250,23.441326
34026,12934525,5278,4.72,0.9062,25.101,0.240,5150.14,0.573,0.4144,0,...,0.149,270628934,13.781,NaN,AAA-222-111-000-0-0,15.720,1.883393,13.327540,1.494237,16.798539
34027,12934557,5341,4.58,0.9227,31.226,0.729,1889.88,0.401,0.2511,0,...,0.103,271409266,12.604,NaN,AAA-222-111-000-0-0,14.494,2.409996,12.956867,1.925928,16.213484
34028,12984138,4960,4.57,0.8239,12.059,0.038,11734.90,1.194,0.6442,0,...,0.126,272097314,13.673,NaN,AAA-222-111-000-0-0,15.950,0.744837,16.190117,0.598039,20.164222


In [13]:
eu = pd.read_csv('exoplanet.eu_catalog.csv')

In [14]:
re_kic = re.compile(r"^\s*(KIC\s(\d+))")
# re_koi = re.compile(r"^\s*(KOI-(\d+))")
def alt_name_cols(row):
    
    new_row = row.copy(deep=False)
    
    new_row['KIC'] = np.nan
#     new_row['KOI'] = np.nan
    
    s_name = str(row['star_name'])
    
    m_kic = re.search(re_kic, s_name)
    if m_kic is not None:
        new_row['KIC'] = int(m_kic.group(2))
            
#     m_koi = re.search(re_koi, s_name)
#     if m_koi is not None:
#         new_row['KOI'] = int(m_koi.group(2))
    
    
    s_alt_names_str = str(row['star_alternate_names'])
    s_alt_name_list = s_alt_names_str.split(",")
    
    for s_alt_name in reversed(s_alt_name_list):
        m_kic = re.search(re_kic, s_alt_name)
        if m_kic is not None:
            new_row['KIC'] = int(m_kic.group(2))
            continue
        
#         m_koi = re.search(re_koi, s_alt_name)
#         if m_koi is not None:
#             new_row['KOI'] = int(m_koi.group(2))
#             continue
            
    return new_row

In [15]:
kic_eu = eu.apply(alt_name_cols, axis=1)
kic_eu.fillna(np.nan)
kic_eu = kic_eu[(np.isfinite(kic_eu['KIC']))]

In [28]:
# eu.loc[eu['# name'] == 'KIC 10068024 b']
eu.loc[eu['# name'] == 'Kepler-186 b', 'star_alternate_names']

2908    2MASS J19543665+4357180, KIC 8120608, KOI-571,...
Name: star_alternate_names, dtype: object

In [29]:
prot_mcq.loc[prot_mcq['KIC'] == 8120608]

,KIC,Teff,log(g),Mass,PRot,e_PRot,RPer,LPH,w,DC,Flag


In [19]:
kic_eu

,# name,planet_status,mass,mass_error_min,mass_error_max,mass_sini,mass_sini_error_min,mass_sini_error_max,radius,radius_error_min,...,star_age,star_age_error_min,star_age_error_max,star_teff,star_teff_error_min,star_teff_error_max,star_detected_disc,star_magnetic_field,star_alternate_names,KIC
1784,KIC 10068024 b,Confirmed,NaN,NaN,NaN,2.0,0.4,0.4,NaN,NaN,...,NaN,NaN,NaN,6118.0,121.00,127.00,NaN,NaN,NaN,10068024.0
1785,KIC 10255705 b,Confirmed,NaN,NaN,NaN,NaN,NaN,NaN,0.650,0.230,...,NaN,NaN,NaN,5030.0,143.00,389.00,NaN,NaN,2MASS 18512491+4722389,10255705.0
1786,KIC 10544976 (AB) b,Confirmed,NaN,NaN,NaN,13.4,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10544976.0
1787,KIC 11152511 b,Confirmed,NaN,NaN,NaN,NaN,NaN,NaN,0.360,0.025,...,NaN,NaN,NaN,5273.0,471.00,145.00,NaN,NaN,NaN,11152511.0
1788,KIC 12454613 b,Confirmed,NaN,NaN,NaN,NaN,NaN,NaN,0.228,0.037,...,NaN,NaN,NaN,5420.0,158.00,331.00,NaN,NaN,NaN,12454613.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4327,Kepler-995 b,Confirmed,NaN,NaN,NaN,NaN,NaN,NaN,0.241,0.015,...,5.01,-3.20,6.05,5206.0,74.71,110.94,NaN,NaN,"2MASS J18582777+3907517, KIC 4035640, KOI-1881...",4035640.0
4328,Kepler-996 b,Confirmed,NaN,NaN,NaN,NaN,NaN,NaN,0.177,0.022,...,3.63,-1.77,1.81,6014.0,152.18,114.40,NaN,NaN,"2MASS J19273906+4132009, KIC 6205228, KOI-1882...",6205228.0
4329,Kepler-997 b,Confirmed,NaN,NaN,NaN,NaN,NaN,NaN,0.121,0.023,...,3.63,-0.69,0.82,6150.0,54.96,62.57,NaN,NaN,"2MASS J19165600+4956201, KIC 11758544, KOI-188...",11758544.0
4330,Kepler-998 b,Confirmed,NaN,NaN,NaN,NaN,NaN,NaN,0.207,0.027,...,3.31,-1.53,1.55,6058.0,165.40,104.50,NaN,NaN,"2MASS J19470099+4912314, KIC 11413812, KOI-188...",11413812.0


In [17]:
kic_mcq_eu = pd.merge(mcq_tic, kic_eu, how='inner', on='KIC')
kic_mcq_eu
# len(kic_mcq_eu)

,KIC,Teff_x,log(g),Mass_x,PRot,e_PRot,RPer,LPH,w,DC,...,star_sp_type,star_age,star_age_error_min,star_age_error_max,star_teff,star_teff_error_min,star_teff_error_max,star_detected_disc,star_magnetic_field,star_alternate_names
0,7350067,3224,5.09,0.2619,3.514,0.006,48054.81,1.540,0.7205,1,...,NaN,4.68,-2.87,5.89,3299.0,41.07,36.22,NaN,NaN,"2MASS J19100633+4254464, KIC 7350067, KOI-6863"
1,7742408,4739,4.07,0.7658,26.630,0.046,8559.17,0.849,0.5410,0,...,NaN,NaN,NaN,NaN,5017.0,178.30,62.80,NaN,NaN,"7742408, 2MASS J19103921+4328172, KIC 7742408,..."
2,7935997,5622,4.54,0.9978,7.577,0.020,7556.97,1.205,0.6705,0,...,NaN,4.27,-2.51,3.20,5776.0,74.65,68.39,NaN,NaN,"2MASS J18404958+4346274, KIC 7935997, KOI-5447..."
3,7978202,4829,4.54,0.7896,29.991,0.153,5300.49,0.738,0.4663,0,...,NaN,5.01,-3.13,5.80,4736.0,64.28,97.30,NaN,NaN,"2MASS J19500199+4342394, KIC 7978202, KOI-5454..."
4,8435766,4957,4.45,0.8231,12.588,0.030,6604.05,0.937,0.5209,0,...,G,NaN,NaN,NaN,5089.0,50.00,50.00,NaN,Yes,"2MASS J19345800+4426539, KIC 8435766, Kepler-7..."
5,8891684,5403,4.23,0.9390,19.794,0.177,8281.11,1.234,0.6338,0,...,NaN,4.47,-2.73,5.70,5474.0,144.37,149.64,NaN,NaN,"2MASS J19361041+4508232, KIC 8891684, KOI-5581..."
6,9413313,5165,4.25,0.8769,14.273,0.030,22919.06,1.133,0.6475,0,...,NaN,NaN,NaN,NaN,5359.0,143.00,167.00,NaN,NaN,NaN
7,10717220,5489,4.54,0.9618,1.451,0.006,6428.96,1.668,0.8108,0,...,NaN,4.27,-2.61,5.87,5499.0,63.10,66.43,NaN,NaN,"2MASS J19015401+4802057, KIC 10717220, KOI-622..."
8,11152511,5314,4.22,0.9156,39.800,0.565,918.84,0.601,0.3150,0,...,NaN,NaN,NaN,NaN,5273.0,471.00,145.00,NaN,NaN,NaN
9,12454613,5335,4.69,0.9211,23.267,0.128,5287.17,0.765,0.4964,0,...,NaN,NaN,NaN,NaN,5420.0,158.00,331.00,NaN,NaN,NaN


In [ ]:
habitable = pd.read_csv("habitable.txt")

In [ ]:
habitable_eu = pd.merge(habitable, eu, on="# name")
# habitable_eu

In [ ]:
set(habitable['# name']) - set(habitable_eu['# name'])

In [ ]:
tempnames = list(habitable['# name'])
def checknames(s):
    for name in tempnames:
        if name[:-2] in s:
            return True
    return False
habitable_eu2 = kic_mcq_eu[kic_mcq_eu.apply(lambda x: checknames(str(x['star_alternate_names'])), axis=1)]
kic_mcq_eu_addl = kic_mcq_eu.iloc[list(habitable_eu2['semi_major_axis'].notnull().index)]
# kic_mcq_eu_addl

In [ ]:
# kic_mcq_eu = pd.merge(kic_mcq_eu, kic_mcq_eu_addl, how='left', on='# name')

In [ ]:
data = tic_mcq_eu[(tic_mcq_eu['RoVK'].notnull() | tic_mcq_eu['RoM'].notnull()) & tic_mcq_eu['semi_major_axis'].notnull()][['# name', 'RoVK', 'RoM', 'semi_major_axis', 'Rad_y', 'mass', 'radius']]
data.fillna(np.nan)
data['RoAvg'] = data.apply(lambda x: RoAvg(x['RoVK'], x['RoM']), axis=1)
data

In [ ]:
np.max(data['RoAvg'])

In [ ]:
habitable_ro = pd.merge(habitable, data)
habitable_ro

In [ ]:
x = np.linspace(0.,2.7,1000) # ro/ro sol x vals
prot_sol = 27.
ro_sol = 1.85
ra_sol = 0.1
r_sol = 1/200.
# all values scaled to solar values at maximum
# combining eqns: (7) Farrish 2019 and (2) Farrish 2021
def ra_schrijver(ro, rad):
    return pow(ro / ro_sol, -1.5 * -0.16) * pow(rad / r_sol, -2 * -0.16)

In [ ]:
fig1 = plt.figure(figsize=(15,8))
ax1 = fig1.add_subplot(111)
ax1.plot(x, ra_schrijver(x * ro_sol, r_sol), color='black', label='Schrijver', linewidth=4)
ax1.scatter(data['RoAvg'] / ro_sol, data['semi_major_axis'] / ra_sol, s=75, label="KIC Planets")
ax1.scatter(habitable_ro['RoAvg'] / ro_sol, habitable_ro['semi_major_axis'] / ra_sol, color='#00FF00', s=150, label="Goldilocks Planets")
ax1.set_title("Orbital Radius vs. Rossby Number",fontsize=25)
ax1.set_xlabel("Ro/RoSun",fontsize=20)
ax1.set_ylabel("RA/RASun",fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
ax1.grid(visible=True)
ax1.set_xlim([0.0,1.5])
ax1.set_ylim([0.0,5.])
ax1.legend(loc=1,fontsize=15)
pass

In [ ]:
data

In [ ]:
alfven = data.apply(lambda x: x['semi_major_axis'] / (ra_sol * ra_schrijver(x['RoAvg'], x['Rad_y'])), axis=1)
alfven_data = data.copy(deep=False)
alfven_data['orbit-alfven'] = alfven
alfven_data = alfven_data[alfven_data['Rad_y'].notnull()]
alfven_data

In [ ]:
alfven_habitable = pd.merge(habitable, alfven_data, on='# name')
alfven_habitable

In [ ]:
alfven_bad = alfven_data[alfven_data['orbit-alfven'] < 0.8]
alfven_maybe = alfven_data[(alfven_data['orbit-alfven'] > 0.8) & (alfven_data['orbit-alfven'] < 1.2)]
alfven_good = alfven_data[alfven_data['orbit-alfven'] > 1.2]
print([len(alfven_bad),len(alfven_maybe),len(alfven_good)])

In [ ]:
fig2 = plt.figure(figsize=(15,8))
ax2 = fig2.add_subplot(111)
ax2.scatter(alfven_bad['RoAvg'],alfven_bad['orbit-alfven'], color='#FF6666', s=75, label='Inside AS')
ax2.scatter(alfven_maybe['RoAvg'],alfven_maybe['orbit-alfven'], color='#DDBB44', s=75, label='Near AS')
ax2.scatter(alfven_good['RoAvg'],alfven_good['orbit-alfven'], color='#66FF66', s=75, label='Outside AS')
for idx, row in alfven_habitable.iterrows():
    xpt = row['RoAvg']
    ypt = row['orbit-alfven']
    lbl = row['# name']
    ax2.scatter(xpt, ypt, color='#0000FF', s=250, marker='*')
#     ax2.text(xpt + 0.02, ypt + 0.01, lbl, fontsize=20)
    
ax2.set_title("Exoplanet Orbits and RA Estimates",fontsize=25)
ax2.set_xlabel("Ro/RoSun",fontsize=20)
ax2.set_ylabel("a/RA",fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
ax2.grid(visible=True)
# ax2.set_xlim([0.0,1.5])
# ax2.set_ylim([0.0,2.0])
ax2.legend(loc=1,fontsize=15)
plt.axhline(y = 1.0, xmin = 0.00, xmax = np.max(data['RoAvg']), linestyle='--', color='#666666')
# plt.text(1.52, 0.98, 'AS', fontsize=15)
plt.text(2.62, 0.98, 'AS', fontsize=15)
pass

In [ ]:
fig3 = plt.figure(figsize=(15,8))
ax3 = fig3.add_subplot(111)
# ax3.plot(x, ra_schrijver(x * ro_sol, r_sol), color='black', label='Schrijver')
ax3.scatter(data['RoAvg'], ra_schrijver(data['RoAvg'], data['Rad_y']), color='#0000FF', label="KIC Planets' Alfven Radius")
ax3.scatter(data['RoAvg'], data['semi_major_axis'] / ra_sol, color='#FF0000', label="KIC Planets' Orbital Radius")
# ax3.scatter(habitable_ro['RoAvg'], alfven[habitable], color='#00FF00', label="Goldilocks Planets")
ax3.set_title("RA vs. Ro",fontsize=25)
ax3.set_xlabel("Ro/RoSun",fontsize=20)
ax3.set_ylabel("RA/RASun",fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
ax3.grid(visible=True)
# ax3.set_xlim([0.0,1.5])
# ax3.set_ylim([0.0,2.5])
ax3.legend(loc=4,fontsize=15)
pass

In [ ]:
# 0.1 to 0.5 ME or 0.4 to 0.8 RE
alfven_subterr = alfven_data[(np.fabs(alfven_data['mass'] - 0.3) <= 0.2) | (np.fabs(alfven_data['radius'] - 0.6) <= 0.2)]
# 0.5 to 3.0 ME or 0.8 to 1.6 RE
alfven_terr = alfven_data[(np.fabs(alfven_data['mass'] - 1.75) <= 1.25) | (np.fabs(alfven_data['radius'] - 1.2) <= 0.4)]
# 3.0 to 10. ME or 1.6 to 2.5 RE
alfven_superterr = alfven_data[(np.fabs(alfven_data['mass'] - 6.5) <= 3.5) | (np.fabs(alfven_data['radius'] - 2.05) <= 0.45)]
# > 10. ME or > 2.5 RE
alfven_giant = alfven_data[(alfven_data['mass'] > 10.) | (alfven_data['radius'] > 2.5)]
print([len(alfven_subterr),len(alfven_terr),len(alfven_superterr), len(alfven_giant)])

In [ ]:
# pd.merge(alfven_subterr, habitable, on='# name')
# pd.merge(alfven_terr, habitable, on='# name')
# pd.merge(alfven_superterr, habitable, on='# name')
# pd.merge(alfven_giant, habitable, on='# name')

In [ ]:
fig4 = plt.figure(figsize=(15,8))
ax4 = fig4.add_subplot(111)
ax4.scatter(alfven_subterr['RoAvg'],alfven_subterr['orbit-alfven'], color='#DDBB44', s=75, label='Subterran')
ax4.scatter(alfven_terr['RoAvg'],alfven_terr['orbit-alfven'], color='#00AA00', s=75, label='Terran')
ax4.scatter(alfven_superterr['RoAvg'],alfven_superterr['orbit-alfven'], color='#FF5500', s=75, label='Superterran')
ax4.scatter(alfven_giant['RoAvg'],alfven_giant['orbit-alfven'], color='#00FFFF', s=75, label='Giant')
# for idx, row in alfven_habitable.iterrows():
#     xpt = row['RoAvg']
#     ypt = row['orbit-alfven']
#     lbl = row['# name']
#     ax2.scatter(xpt, ypt, color='#0000FF', s=250, marker='*')
#     ax2.text(xpt + 0.02, ypt + 0.01, lbl, fontsize=20)
    
ax4.set_title("Exoplanet Orbits and RA Estimates",fontsize=25)
ax4.set_xlabel("Ro/RoSun",fontsize=20)
ax4.set_ylabel("a/RA",fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
ax4.grid(visible=True)
# ax4.set_xlim([0.0,1.5])
# ax4.set_ylim([0.0,2.0])
ax4.legend(loc=1,fontsize=15)
plt.axhline(y = 1.0, xmin = 0.00, xmax = np.max(data['RoAvg']), linestyle='--', color='#666666')
# plt.text(1.52, 0.98, 'AS', fontsize=15)
plt.text(2.62, 0.98, 'AS', fontsize=15)
pass